# Setup the Env

In [1]:
import sys
import os

import sagemaker
from sagemaker import get_execution_role

# Add the parent directory to the sys.path
sys.path.insert(0, os.path.abspath(".."))

# Define IAM role
role = get_execution_role()
role

# Establish S3 bucket connection
import boto3

s3 = boto3.client("s3")
bucket = "capstone-bucket-4-friends"

# Take a look at current dir
print(os.getcwd())

from file_utilities import s3_download

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
/home/sagemaker-user/capstone-2024-summer/src/rachel


# Import libraries

In [2]:
# standard libraries
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt

!pip install seaborn -q
import seaborn as sns

# Download Data from S3 as needed

In [3]:
# crsp = "CRSP/crsp_2018-2023.csv"
# crsp_path = s3_download(crsp)

In [4]:
# index = "Index/security_master.csv"
# index_path = s3_download(index)

# Load the dataset

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Stock_&_Indexes_Database_Data_Descriptions_Guide.pdf

In [5]:
dtype_dict = {
    "permno_id": "object",
    "ncusip_id": "object",
    "cusip_id": "object",
    "ticker": "object",
    "share_code": "object",
    "share_code_type": "object",
    "share_code_detail": "object",
    "exchange_code": "object",
    "company_name": "object",
    "primary_exchange": "object",
    "trading_status": "object",
    "security_status": "object",
    "naics": "object",
    "naics_sector": "object",
    "naics_secondary": "object",
    "ask_or_high_price": "float64",
    "ask_price": "float64",
    "bid_or_low_price": "float64",
    "bid_price": "float64",
    "open_price": "float64",
    "close_price": "float64",
    "return": "float64",
    "volume": "int64",
    "shares_outstanding": "int64",
    "num_trades": "int64",
    "factor_to_adjust_price": "float64",
    "factor_to_adjust_shares": "float64",
    "value_weighted_return": "float64",
    "equal_weighted_return": "float64",
    "return_on_SP_index": "float64",
    "log_return": "float64",
    "market_cap": "float64",
    "weighted_price": "float64",
    "sector_weighted_avg_log_return": "float64",
    "sector_simple_avg_log_return": "float64",
    "weighted_avg_log_return": "float64",
    "simple_avg_log_return": "float64",
    "price_7": "float64",
    "return_7": "float64",
    "log_return_7": "float64",
    "abs_log_return_7": "float64",
    "price_14": "float64",
    "return_14": "float64",
    "log_return_14": "float64",
    "abs_log_return_14": "float64",
    "price_28": "float64",
    "return_28": "float64",
    "log_return_28": "float64",
    "abs_log_return_28": "float64",
    "volatility_7": "float64",
    "volatility_14": "float64",
    "volatility_28": "float64",
}

In [6]:
crsp_df = pd.read_csv(
    "/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_2.csv",
    dtype=dtype_dict,
    parse_dates=["date"],
)

crsp_df.head()

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,return_14,log_return_14,abs_log_return_14,price_28,return_28,log_return_28,abs_log_return_28,volatility_7,volatility_14,volatility_28
0,2018-01-02,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.125487,0.118216,0.118216,134.24001,-0.901326,-2.315935,2.315935,0.160708,0.251431,0.322190
1,2018-01-03,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.081658,0.078495,0.078495,134.61000,-0.797075,-1.594919,1.594919,0.151850,0.253598,0.322152
2,2018-01-04,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.143058,0.133707,0.133707,138.36000,-0.683430,-1.150211,1.150211,0.114113,0.257447,0.331450
3,2018-01-05,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.052192,0.050875,0.050875,138.78000,-0.577748,-0.862153,0.862153,0.250402,0.262128,0.331206
4,2018-01-08,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.138266,0.129506,0.129506,139.10001,-0.564584,-0.831453,0.831453,0.264054,0.263091,0.331376


In [7]:
security_df = pd.read_csv("/home/sagemaker-user/capstone-2024-summer/data/security_master.csv")

security_df.head()

,cusip,PERMNO,PERMCO,HSHRCD,DLSTCD,HTICK,HCOMNAM,HTSYMBOL,HNAICS,HPRIMEXC,...,NUMDEL,NUMNDI,BEGDAT,ENDDAT,BEGPRC,ENDPRC,BEGRET,ENDRET,BEGVOL,ENDVOL
0,00130H10,76712,10996,11,100,AES,A E S CORP,AES,221118,N,...,1,68,06/26/1991,12/29/2023,06/26/1991,12/29/2023,06/26/1991,12/29/2023,06/26/1991,12/29/2023
1,00206R10,66093,21645,11,100,T,A T & T INC,T,517312,N,...,1,0,02/16/1984,12/29/2023,02/16/1984,12/29/2023,02/16/1984,12/29/2023,02/16/1984,12/29/2023
2,00507V10,79678,12499,11,233,NaN,ACTIVISION BLIZZARD INC,ATVI,513210,Q,...,1,1968,10/22/1993,10/12/2023,10/22/1993,10/12/2023,10/22/1993,10/12/2023,10/22/1993,10/12/2023
3,00724F10,75510,8476,11,100,ADBE,ADOBE INC,ADBE,511210,Q,...,1,2185,08/13/1986,12/29/2023,08/13/1986,12/29/2023,08/13/1986,12/29/2023,08/13/1986,12/29/2023
4,00971T10,87299,17300,11,100,AKAM,AKAMAI TECHNOLOGIES INC,AKAM,511210,Q,...,1,1722,10/29/1999,12/29/2023,10/29/1999,12/29/2023,10/29/1999,12/29/2023,10/29/1999,12/29/2023


# Take a look at the columns

In [8]:
crsp_df.columns

Index(['date', 'permno_id', 'ncusip_id', 'cusip_id', 'ticker', 'share_code',
       'share_code_type', 'share_code_detail', 'exchange_code', 'company_name',
       'primary_exchange', 'trading_status', 'security_status', 'naics',
       'naics_sector', 'naics_secondary', 'ask_or_high_price', 'ask_price',
       'bid_or_low_price', 'bid_price', 'open_price', 'close_price', 'return',
       'volume', 'shares_outstanding', 'num_trades', 'factor_to_adjust_price',
       'factor_to_adjust_shares', 'value_weighted_return',
       'equal_weighted_return', 'return_on_SP_index', 'log_return',
       'abs_log_return', 'market_cap', 'weighted_price',
       'sector_weighted_avg_log_return', 'sector_simple_avg_log_return',
       'weighted_avg_log_return', 'simple_avg_log_return', 'price_7',
       'return_7', 'log_return_7', 'abs_log_return_7', 'price_14', 'return_14',
       'log_return_14', 'abs_log_return_14', 'price_28', 'return_28',
       'log_return_28', 'abs_log_return_28', 'volatility_7'

In [9]:
crsp_df.dtypes

date                              datetime64[ns]
permno_id                                 object
ncusip_id                                 object
cusip_id                                  object
ticker                                    object
share_code                                object
share_code_type                           object
share_code_detail                         object
exchange_code                             object
company_name                              object
primary_exchange                          object
trading_status                            object
security_status                           object
naics                                     object
naics_sector                              object
naics_secondary                           object
ask_or_high_price                        float64
ask_price                                float64
bid_or_low_price                         float64
bid_price                                float64
open_price          

In [10]:
og_crsp_len = len(crsp_df)
og_crsp_cols = len(crsp_df.columns)
print("number of records:", og_crsp_len)
print("number of columns:", og_crsp_cols)

number of records: 7658922
number of columns: 54


## A. Keep only PERMNO in security master

In [11]:
PERMNO = security_df["PERMNO"].astype(str).unique()
len(PERMNO)

594

In [12]:
og_len = len(crsp_df)
crsp_df = crsp_df[crsp_df["permno_id"].isin(PERMNO)]
new_len = len(crsp_df)
print("num_records_removed: ", og_len - new_len)
print("num_records_kept: ", new_len)

num_records_removed:  6849039
num_records_kept:  809883


In [13]:
len(crsp_df["permno_id"].unique())

554

# Check a few large companies to ensure they are still in the dataset

["GOOG", "NVDA", "TSLA", "AMZN"]

In [19]:
assert all(ticker in crsp_df["ticker"].unique() for ticker in ["GOOG", "NVDA", "TSLA", "AMZN"])

# Confirm each PERMNO contains at most num_days records (each PERMNO only has one record per day)

In [20]:
assert crsp_df["permno_id"].value_counts().max() <= crsp_df["date"].nunique()

# Take a final look at the columns

In [15]:
crsp_df.columns

Index(['date', 'permno_id', 'ncusip_id', 'cusip_id', 'ticker', 'share_code',
       'share_code_type', 'share_code_detail', 'exchange_code', 'company_name',
       'primary_exchange', 'trading_status', 'security_status', 'naics',
       'naics_sector', 'naics_secondary', 'ask_or_high_price', 'ask_price',
       'bid_or_low_price', 'bid_price', 'open_price', 'close_price', 'return',
       'volume', 'shares_outstanding', 'num_trades', 'factor_to_adjust_price',
       'factor_to_adjust_shares', 'value_weighted_return',
       'equal_weighted_return', 'return_on_SP_index', 'log_return',
       'abs_log_return', 'market_cap', 'weighted_price',
       'sector_weighted_avg_log_return', 'sector_simple_avg_log_return',
       'weighted_avg_log_return', 'simple_avg_log_return', 'price_7',
       'return_7', 'log_return_7', 'abs_log_return_7', 'price_14', 'return_14',
       'log_return_14', 'abs_log_return_14', 'price_28', 'return_28',
       'log_return_28', 'abs_log_return_28', 'volatility_7'

In [16]:
crsp_df.dtypes

date                              datetime64[ns]
permno_id                                 object
ncusip_id                                 object
cusip_id                                  object
ticker                                    object
share_code                                object
share_code_type                           object
share_code_detail                         object
exchange_code                             object
company_name                              object
primary_exchange                          object
trading_status                            object
security_status                           object
naics                                     object
naics_sector                              object
naics_secondary                           object
ask_or_high_price                        float64
ask_price                                float64
bid_or_low_price                         float64
bid_price                                float64
open_price          

In [17]:
crsp_len = len(crsp_df)
crsp_cols = len(crsp_df.columns)
print("number of records:", crsp_len)
print("number of columns:", crsp_cols)

number of records: 809883
number of columns: 54


# Save the cleaned df as csv & parquet

In [21]:
crsp_df.to_csv(
    "/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.csv",
    index=False,
)
crsp_df.to_parquet("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet")

In [22]:
reload = pd.read_parquet("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet")
assert crsp_df.equals(reload)

reload.head()

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,return_14,log_return_14,abs_log_return_14,price_28,return_28,log_return_28,abs_log_return_28,volatility_7,volatility_14,volatility_28
9263,2018-01-02,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.733219,1.005480,1.005480,48.15,0.293373,0.257254,0.257254,0.136136,0.109282,0.276310
9264,2018-01-03,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.411025,0.880052,0.880052,48.49,0.147139,0.137271,0.137271,0.091648,0.081082,0.268724
9265,2018-01-04,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.277709,0.823170,0.823170,49.45,0.237235,0.212879,0.212879,0.089939,0.079524,0.273385
9266,2018-01-05,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.589437,0.951440,0.951440,50.64,0.402930,0.338563,0.338563,0.105015,0.107098,0.281463
9267,2018-01-08,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.087791,0.736107,0.736107,50.71,0.317885,0.276028,0.276028,0.105560,0.139328,0.280139


# Upload the parquet to s3

In [23]:
s3.upload_file(
    "/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet",
    bucket,
    "CRSP/crsp_2018-2023_clean_3.parquet",
)

In [24]:
reload_path = s3_download("CRSP/crsp_2018-2023_clean_3.parquet")

In [25]:
reload = pd.read_parquet("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet")
assert crsp_df.equals(reload)

reload.head()

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,return_14,log_return_14,abs_log_return_14,price_28,return_28,log_return_28,abs_log_return_28,volatility_7,volatility_14,volatility_28
9263,2018-01-02,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.733219,1.005480,1.005480,48.15,0.293373,0.257254,0.257254,0.136136,0.109282,0.276310
9264,2018-01-03,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.411025,0.880052,0.880052,48.49,0.147139,0.137271,0.137271,0.091648,0.081082,0.268724
9265,2018-01-04,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.277709,0.823170,0.823170,49.45,0.237235,0.212879,0.212879,0.089939,0.079524,0.273385
9266,2018-01-05,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.589437,0.951440,0.951440,50.64,0.402930,0.338563,0.338563,0.105015,0.107098,0.281463
9267,2018-01-08,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,1.087791,0.736107,0.736107,50.71,0.317885,0.276028,0.276028,0.105560,0.139328,0.280139
